### Paso 1: .doc a .csv

In [2]:
import os
import re
from docx import Document
import pandas as pd

In [3]:
# Ruta a carpeta local con archivos .docx y donde se guardara el archivo .csv
carpeta = "Informes/"
archivo_csv = "documentos_medicos.csv"
datos = []
nuevos_datos = []

In [4]:
def extraer_texto_docx(archivo):
    doc = Document(archivo)
    texto = '\n'.join([p.text for p in doc.paragraphs if p.text.strip() != ""])
    return texto

In [5]:
def extraer_campos(texto):
    paciente = re.search(r"Paciente:\s*(.*)", texto)
    edad = re.search(r"Edad:\s*(\d+)", texto)
    fecha = re.search(r"Fecha de Ingreso:\s*(.*)", texto)
    dx = re.search(r"Diagnóstico Presuntivo:\s*(.*)", texto)
    meds = re.search(r"Medicamentos Recetados:(.*?)(Instrucciones|Firma|$)", texto, re.DOTALL)

    return {
        "paciente": paciente.group(1).strip() if paciente else "",
        "edad": int(edad.group(1)) if edad else "",
        "fecha_ingreso": fecha.group(1).strip() if fecha else "",
        "diagnostico": dx.group(1).strip() if dx else "",
        "medicamentos": meds.group(1).strip().replace("\n", ", ") if meds else ""
    }

In [6]:
# Cargar CSV existente si ya existe
if os.path.exists(archivo_csv):
    df_existente = pd.read_csv(archivo_csv, encoding='utf-8')
else:
    df_existente = pd.DataFrame()

In [7]:
for archivo in os.listdir(carpeta):
    if archivo.endswith(".docx"):
        ruta = os.path.join(carpeta, archivo)
        texto = extraer_texto_docx(ruta)
        campos = extraer_campos(texto)
        campos["archivo"] = archivo
        datos.append(campos)

        # Verificar si el paciente existe
        if not df_existente.empty and campos["paciente"] in df_existente["paciente"].values:
            print(f"Paciente ya registrado: {campos['paciente']} — ignorado.")
        else:
            nuevos_datos.append(campos)

# Agregar y guardar
if nuevos_datos:
    df_nuevos = pd.DataFrame(nuevos_datos)
    df_final = pd.concat([df_existente, df_nuevos], ignore_index=True)
    df_final.to_csv(archivo_csv, index=False, encoding='utf-8')
    print("Datos nuevos agregados al CSV.")
else:
    print("No se encontraron pacientes nuevos.")

Paciente ya registrado: Juan Pérez — ignorado.
No se encontraron pacientes nuevos.


In [8]:
# Guardar en CSV
df = pd.DataFrame(datos)
df.to_csv("documentos_medicos.csv", index=False, encoding='utf-8')

# Mostrar el contenido del CSV
print(df.head())

     paciente  edad        fecha_ingreso                 diagnostico  \
0  Juan Pérez    45  15 de abril de 2025  Posible apendicitis aguda.   

                                        medicamentos    archivo  
0  Paracetamol 500mg: 1 tableta cada 8 horas para...  tets.docx  


In [9]:
# Leer el archivo CSV
df = pd.read_csv("documentos_medicos.csv", encoding='utf-8')

# Mostrar las primeras filas
print(df.head())

     paciente  edad        fecha_ingreso                 diagnostico  \
0  Juan Pérez    45  15 de abril de 2025  Posible apendicitis aguda.   

                                        medicamentos    archivo  
0  Paracetamol 500mg: 1 tableta cada 8 horas para...  tets.docx  


### Paso 2: Entrenar modelos

#### Modelo GPT2

In [10]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Cargar el modelo y el tokenizer de GPT-2
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Ajustar el token de padding
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

In [11]:
def generar_epicrisis_gpt2(texto_informe, max_new_tokens=200):
    prompt = f"Resumen médico del siguiente informe clínico:\n{texto_informe}\n\nEpicrisis:"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        num_beams=4,
        no_repeat_ngram_size=2,
        early_stopping=True,
        pad_token_id=tokenizer.pad_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [25]:
def guardar_epicrisis_docx(nombre_paciente, texto_epicrisis):
    nombre_archivo = f"epicrisis_{nombre_paciente.replace(' ', '_')}_gpt2.docx"

    if os.path.exists(nombre_archivo):
        print("")
    else:
        print(f"Epicrisis generada en epicrisis_{nombre_paciente.replace(' ', '_')}_gpt2.docx")
        
    doc = Document()
    doc.add_heading("Epicrisis", level=1)
    doc.add_paragraph(f"Paciente: {nombre_paciente}", style='Normal')
    doc.add_paragraph(texto_epicrisis, style='Normal')
    
    doc.save(nombre_archivo)

In [26]:
epicrisis_resultados = []

for archivo in os.listdir(carpeta):
    if archivo.endswith(".docx"):
        ruta = os.path.join(carpeta, archivo)
        texto = extraer_texto_docx(ruta)
        campos = extraer_campos(texto)
        campos["archivo"] = archivo

        # Generar epicrisis
        epicrisis = generar_epicrisis_gpt2(texto)
        campos["epicrisis"] = epicrisis
        epicrisis_resultados.append({"paciente": campos["paciente"], "epicrisis": epicrisis})

        # Guardar en archivo
        guardar_epicrisis_docx(campos["paciente"], epicrisis)

Epicrisis generada en epicrisis_Juan_Pérez_gpt2.docx


#### microsoft/BioGPT-Large

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "microsoft/BioGPT-Large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

#### stanford-biomed/biomedgpt-1.3b

In [ ]:
model_name = "stanford-biomed/biomedgpt-1.3b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
# Borrar csv al final